In [ ]:
'''
Save and Restore a model using TensorFlow.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [2]:
import numpy as np
import tensorflow as tf
import read_mnist_data as rmd

# read dataset from disk
DATA_FOLDER = '/home/chengming/TensorFlow-Examples/data/'
TRAIN_IMAGES_SET = 'train-images-idx3-ubyte.gz'
TRAIN_LABELS_SET = 'train-labels-idx1-ubyte.gz'
TEST_IMAGES_SET = 't10k-images-idx3-ubyte.gz'
TEST_LABELS_SET = 't10k-labels-idx1-ubyte.gz'
validation_size = 5000
with open(DATA_FOLDER + TRAIN_IMAGES_SET, 'rb') as f:
    train_images = rmd.extract_images(f)
with open(DATA_FOLDER + TRAIN_LABELS_SET, 'rb') as f:
    train_labels = rmd.extract_labels(f, one_hot = True)
with open(DATA_FOLDER + TEST_IMAGES_SET, 'rb') as f:
    test_images = rmd.extract_images(f)
with open(DATA_FOLDER + TEST_LABELS_SET, 'rb') as f:
    test_labels = rmd.extract_labels(f, one_hot = True)
#validation_images = train_images[:validation_size]
#validation_labels = train_labels[:validation_size]
#train_images = train_images[validation_size:]
#train_labels = train_labels[validation_size:]

train_shape = train_images.shape
test_shape = test_images.shape
train_images = train_images.reshape(train_shape[0], train_shape[1]*train_shape[2])
test_images = test_images.reshape(test_shape[0], test_shape[1]*test_shape[2])


print train_images.shape
print train_labels.shape, '\t', train_labels[0][0:6], train_labels[0][6:]
print test_images.shape
print test_labels.shape, '\t', test_labels[0]

('Extracting', '/home/chengming/TensorFlow-Examples/data/train-images-idx3-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/train-labels-idx1-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/t10k-images-idx3-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/t10k-labels-idx1-ubyte.gz')
(60000, 784)
(60000, 10) 	[ 0.  0.  0.  0.  0.  1.] [ 0.  0.  0.  0.]
(10000, 784)
(10000, 10) 	[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


In [8]:
# Parameters
learning_rate = 0.001
batch_size = 100
display_step = 1
model_path = "/home/chengming/TensorFlow-Examples/notebooks/4_Utils/model.ckpt"

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [9]:
# 'Saver' op to save and restore all the variables
# create a Saver
saver = tf.train.Saver()

In [10]:
# Running first session
print "Starting 1st session..."
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    # Initialize variables
    sess.run(init)

    # Training cycle
    for epoch in range(3):
        avg_cost = 0.
        #total_batch = int(mnist.train.num_examples/batch_size)
        total_batch = int(train_images.shape[0] / batch_size)
        start_idx = 0
        # Loop over all batches
        for i in range(total_batch):
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            batch_x = train_images[start_idx:start_idx+batch_size, :]
            batch_y = train_labels[start_idx:start_idx+batch_size, :]
            start_idx += batch_size
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "First Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: test_images, y: test_labels})

    # Save model weights to disk
    # tf.train.Saver.save(sess, save_path, global_step=None, 
    #                     latest_filename=None, meta_graph_suffix='meta', 
    #                     write_meta_graph=True, write_state=True)
    save_path = saver.save(sess, model_path)
    print "Model saved in file: %s" % save_path

Starting 1st session...
Epoch: 0001 cost= 41900.840388997
Epoch: 0002 cost= 9914.031634064
Epoch: 0003 cost= 6266.206561165
First Optimization Finished!
Accuracy: 0.9166
Model saved in file: /home/chengming/TensorFlow-Examples/notebooks/4_Utils/model.ckpt


In [11]:
# Running a new session
print "Starting 2nd session..."
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    # Initialize variables
    sess.run(init)

    # Restore model weights from previously saved model
    # tf.train.Saver.restore(sess, save_path)
    load_path = saver.restore(sess, model_path)
    print "Model restored from file: %s" % save_path

    # Resume training
    for epoch in range(7):
        avg_cost = 0.
        #total_batch = int(mnist.train.num_examples / batch_size)
        total_batch = int(train_images.shape[0] / batch_size)
        start_idx = 0
        # Loop over all batches
        for i in range(total_batch):
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            batch_x = train_images[start_idx:start_idx+batch_size, :]
            batch_y = train_labels[start_idx:start_idx+batch_size, :]
            start_idx += batch_size
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch + 1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Second Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: test_images, y: test_labels})

Starting 2nd session...
Model restored from file: /home/chengming/TensorFlow-Examples/notebooks/4_Utils/model.ckpt
Epoch: 0001 cost= 4390.139832290
Epoch: 0002 cost= 3202.973835990
Epoch: 0003 cost= 2354.944862804
Epoch: 0004 cost= 1759.677807693
Epoch: 0005 cost= 1306.321815287
Epoch: 0006 cost= 1018.067773261
Epoch: 0007 cost= 772.719040394
Second Optimization Finished!
Accuracy: 0.9375
